In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing

import xgboost as xgb
from sklearn import svm

pd.set_option('display.max_columns', None)

In [2]:
# Cárrega de la base de dades
db_vols = pd.read_csv('H:\_vertex\_formacio\_python - cibernarium\_12\DelayedFlights.csv')

# Donar estils a les taules
db_vols.style.set_table_styles([{'selector' : '','props' : [('border','5px solid green')]}])

db_vols.head()

,Unnamed: 0,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0,N,0,NaN,NaN,NaN,NaN,NaN
1,1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
2,2,2008,1,3,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,0,N,0,NaN,NaN,NaN,NaN,NaN
3,4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0
4,5,2008,1,3,4,1940.0,1915,2121.0,2110,WN,378,N726SW,101.0,115.0,87.0,11.0,25.0,IND,JAX,688,4.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN


In [3]:
# Faig una cópia per evitar de reexecutar tantes vegades 
vols = db_vols.copy()

In [4]:
# Eliminar columna anomenada Unnamed: 0
vols = vols.rename(columns={"Unnamed: 0": "id"})

# Fixar com a index la columna id
vols = vols.set_index('id')

# Elimina la columna index
#vols.drop('index', axis='columns', inplace=True)

vols.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0,N,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,0,N,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0
5,2008,1,3,4,1940.0,1915,2121.0,2110,WN,378,N726SW,101.0,115.0,87.0,11.0,25.0,IND,JAX,688,4.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN


In [5]:
# Mida del dataframe
vols.shape

(1936758, 29)

In [6]:
# Fer cop d'ull als valors NULL
vols.isna().sum()

Year                      0
Month                     0
DayofMonth                0
DayOfWeek                 0
DepTime                   0
CRSDepTime                0
ArrTime                7110
CRSArrTime                0
UniqueCarrier             0
FlightNum                 0
TailNum                   5
ActualElapsedTime      8387
CRSElapsedTime          198
AirTime                8387
ArrDelay               8387
DepDelay                  0
Origin                    0
Dest                      0
Distance                  0
TaxiIn                 7110
TaxiOut                 455
Cancelled                 0
CancellationCode          0
Diverted                  0
CarrierDelay         689270
WeatherDelay         689270
NASDelay             689270
SecurityDelay        689270
LateAircraftDelay    689270
dtype: int64

In [7]:
# Elimino les columnes amb tants valors NULL
vols = vols.drop(['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'], axis=1)

In [8]:
# Comprobació de que efectivamnent s'han eliminat aquestes columnes
vols.isna().sum()

Year                    0
Month                   0
DayofMonth              0
DayOfWeek               0
DepTime                 0
CRSDepTime              0
ArrTime              7110
CRSArrTime              0
UniqueCarrier           0
FlightNum               0
TailNum                 5
ActualElapsedTime    8387
CRSElapsedTime        198
AirTime              8387
ArrDelay             8387
DepDelay                0
Origin                  0
Dest                    0
Distance                0
TaxiIn               7110
TaxiOut               455
Cancelled               0
CancellationCode        0
Diverted                0
dtype: int64

In [9]:
# Potser fora convenient eliminar les files amb dades amb valor NULL
vols = vols.dropna(axis=0, how='any')
vols.isna().sum()

Year                 0
Month                0
DayofMonth           0
DayOfWeek            0
DepTime              0
CRSDepTime           0
ArrTime              0
CRSArrTime           0
UniqueCarrier        0
FlightNum            0
TailNum              0
ActualElapsedTime    0
CRSElapsedTime       0
AirTime              0
ArrDelay             0
DepDelay             0
Origin               0
Dest                 0
Distance             0
TaxiIn               0
TaxiOut              0
Cancelled            0
CancellationCode     0
Diverted             0
dtype: int64

In [10]:
# Mida nova del dataframe
vols.shape

(1928368, 24)

In [11]:
# Estadístics de resum de cada columna categórica
vols.describe(include='object')

,UniqueCarrier,TailNum,Origin,Dest,CancellationCode
count,1928368,1928368,1928368,1928368,1928368
unique,20,5360,303,302,1
top,WN,N325SW,ATL,ORD,N
freq,376201,961,131213,108265,1928368


In [12]:
# Estadístics de resum de cada columna numérica
vols.describe().round(2)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Distance,TaxiIn,TaxiOut,Cancelled,Diverted
count,1928368.0,1928368.00,1928368.00,1928368.00,1928368.00,1928368.00,1928368.00,1928368.00,1928368.00,1928368.00,1928368.00,1928368.00,1928368.00,1928368.00,1928368.00,1928368.00,1928368.00,1928368.0,1928368.0
mean,2008.0,6.11,15.75,3.98,1518.65,1467.72,1610.24,1634.20,2184.29,133.31,134.20,108.28,42.20,43.09,764.95,6.81,18.22,0.0,0.0
std,0.0,3.48,8.78,2.00,450.44,424.73,548.00,464.63,1944.45,72.06,71.23,68.64,56.78,53.27,573.89,5.27,14.31,0.0,0.0
min,2008.0,1.00,1.00,1.00,1.00,0.00,1.00,0.00,1.00,14.00,-21.00,0.00,-109.00,6.00,11.00,0.00,0.00,0.0,0.0
25%,2008.0,3.00,8.00,2.00,1203.00,1135.00,1316.00,1325.00,611.00,80.00,82.00,58.00,9.00,12.00,338.00,4.00,10.00,0.0,0.0
50%,2008.0,6.00,16.00,4.00,1545.00,1510.00,1715.00,1705.00,1543.00,116.00,116.00,90.00,24.00,24.00,606.00,6.00,14.00,0.0,0.0
75%,2008.0,9.00,23.00,6.00,1900.00,1815.00,2030.00,2014.00,3423.00,165.00,165.00,137.00,56.00,53.00,997.00,8.00,21.00,0.0,0.0
max,2008.0,12.00,31.00,7.00,2400.00,2359.00,2400.00,2359.00,9741.00,1114.00,660.00,1091.00,2461.00,2467.00,4962.00,240.00,422.00,0.0,0.0


In [13]:
# Converteixo a categórica la variable número de vol
vols['FlightNum'] = vols['FlightNum'].apply(str)

In [14]:
# Estadístics de resum de cada columna categórica de nou
vols.describe(include='object')

,UniqueCarrier,FlightNum,TailNum,Origin,Dest,CancellationCode
count,1928368,1928368,1928368,1928368,1928368,1928368
unique,20,7498,5360,303,302,1
top,WN,16,N325SW,ATL,ORD,N
freq,376201,1575,961,131213,108265,1928368


In [15]:
# Elimino columnes per enir només un valor
vols = vols.drop(['CancellationCode', 'Year', 'Cancelled', 'Diverted'], axis=1)

In [16]:
# Elimino columnes que no serveixen per a fer regressions numériques
vols = vols.drop(['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime'], axis=1)

In [17]:
# Elimino columnes que no serveixen per a fer regressions numériques
vols = vols.drop(['CRSElapsedTime'], axis=1)

In [18]:
# Estadístics de resum de cada columna categórica de nou
vols.describe(include='object')

,UniqueCarrier,FlightNum,TailNum,Origin,Dest
count,1928368,1928368,1928368,1928368,1928368
unique,20,7498,5360,303,302
top,WN,16,N325SW,ATL,ORD
freq,376201,1575,961,131213,108265


In [19]:
# Estadístics de resum de cada columna numérica de nou
vols.describe().round(2)

,ActualElapsedTime,AirTime,ArrDelay,DepDelay,Distance,TaxiIn,TaxiOut
count,1928368.00,1928368.00,1928368.00,1928368.00,1928368.00,1928368.00,1928368.00
mean,133.31,108.28,42.20,43.09,764.95,6.81,18.22
std,72.06,68.64,56.78,53.27,573.89,5.27,14.31
min,14.00,0.00,-109.00,6.00,11.00,0.00,0.00
25%,80.00,58.00,9.00,12.00,338.00,4.00,10.00
50%,116.00,90.00,24.00,24.00,606.00,6.00,14.00
75%,165.00,137.00,56.00,53.00,997.00,8.00,21.00
max,1114.00,1091.00,2461.00,2467.00,4962.00,240.00,422.00


In [20]:
# Aspecte final del dataframe
vols.head()

,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut
id,,,,,,,,,,,,
0,WN,335,N712SW,128.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0
1,WN,3231,N772SW,128.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0
2,WN,448,N428WN,96.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0
4,WN,3920,N464WN,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0
5,WN,378,N726SW,101.0,87.0,11.0,25.0,IND,JAX,688,4.0,10.0


In [21]:
# Mida final del dataframe
vols.shape

(1928368, 12)

## Nivell 1

- Exercici 1

    Crea almenys tres models de classificació diferents per intentar predir el millor possible l’endarreriment dels vols (ArrDelay) de DelayedFlights.csv. Considera si el vol ha arribat tard o no (ArrDelay > 0).

In [22]:
# Creem la columna 'ArrDelay_delayed' amb valors 0 o 1
vols['ArrDelay_delayed'] = np.where((vols['ArrDelay'] > 0), 1, 0)
display(vols.head())

,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,ArrDelay_delayed
id,,,,,,,,,,,,,
0,WN,335,N712SW,128.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0
1,WN,3231,N772SW,128.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,1
2,WN,448,N428WN,96.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,1
4,WN,3920,N464WN,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,1
5,WN,378,N726SW,101.0,87.0,11.0,25.0,IND,JAX,688,4.0,10.0,1


In [23]:
# Obviem ArrDelay
vols = vols.drop('ArrDelay',axis=1)

In [24]:
# Dimensió de la taula
vols.shape

(1928368, 12)

In [25]:
# Estadistics
vols.describe().round()

,ActualElapsedTime,AirTime,DepDelay,Distance,TaxiIn,TaxiOut,ArrDelay_delayed
count,1928368.0,1928368.0,1928368.0,1928368.0,1928368.0,1928368.0,1928368.0
mean,133.0,108.0,43.0,765.0,7.0,18.0,1.0
std,72.0,69.0,53.0,574.0,5.0,14.0,0.0
min,14.0,0.0,6.0,11.0,0.0,0.0,0.0
25%,80.0,58.0,12.0,338.0,4.0,10.0,1.0
50%,116.0,90.0,24.0,606.0,6.0,14.0,1.0
75%,165.0,137.0,53.0,997.0,8.0,21.0,1.0
max,1114.0,1091.0,2467.0,4962.0,240.0,422.0,1.0


In [26]:
# Per tal d'estalviar temps de calcul primer agafo una mostra aleatoria simple
mostra = 15000
vols = vols.sample(mostra,random_state=1357)

In [27]:
# Variable dependent
y = vols['ArrDelay_delayed']

# Variables predictores
x = vols.drop(['UniqueCarrier', 'FlightNum', 'TailNum', 'Origin', 'Dest', 'ArrDelay_delayed'], axis = 1)

# LLista amb el nom dels predictors
x_columnes = list(x.columns)

In [28]:
print(x_columnes)

['ActualElapsedTime', 'AirTime', 'DepDelay', 'Distance', 'TaxiIn', 'TaxiOut']


In [29]:
# Creació de la mostra d'entrenament i validació
#seed(1234)
mostra_entrenament_x, mostra_validacio_x, mostra_entrenament_y, mostra_validacio_y = train_test_split(x, y, test_size = 0.2, random_state=5678)

mostra_entrenament = pd.concat([mostra_entrenament_x, mostra_entrenament_y], axis=1)
mostra_validacio = pd.concat([mostra_validacio_x, mostra_validacio_y], axis=1)

print("Tamany de mostra d'entrenament: {}".format(mostra_entrenament_x.shape[0]))
print("Tamany de mostra de validació: {}".format(mostra_validacio_x.shape[0]))

display(mostra_entrenament)
display(mostra_entrenament_x.head())
display(mostra_entrenament_y.head())

display(mostra_validacio)
display(mostra_validacio_x.head())
display(mostra_validacio_y.head())

Tamany de mostra d'entrenament: 12000
Tamany de mostra de validació: 3000


,ActualElapsedTime,AirTime,DepDelay,Distance,TaxiIn,TaxiOut,ArrDelay_delayed
id,,,,,,,
6509369,111.0,97.0,10.0,546,4.0,10.0,1
3207192,70.0,18.0,79.0,67,9.0,43.0,1
6730678,62.0,47.0,31.0,328,7.0,8.0,1
54296,91.0,74.0,9.0,507,5.0,12.0,0
632081,123.0,106.0,6.0,810,5.0,12.0,1
...,...,...,...,...,...,...,...
4842848,157.0,142.0,62.0,1240,4.0,11.0,1
6983483,357.0,331.0,49.0,2133,5.0,21.0,1
2224274,166.0,136.0,10.0,1087,6.0,24.0,1


,ActualElapsedTime,AirTime,DepDelay,Distance,TaxiIn,TaxiOut
id,,,,,,
6509369,111.0,97.0,10.0,546,4.0,10.0
3207192,70.0,18.0,79.0,67,9.0,43.0
6730678,62.0,47.0,31.0,328,7.0,8.0
54296,91.0,74.0,9.0,507,5.0,12.0
632081,123.0,106.0,6.0,810,5.0,12.0


id
6509369    1
3207192    1
6730678    1
54296      0
632081     1
Name: ArrDelay_delayed, dtype: int32

,ActualElapsedTime,AirTime,DepDelay,Distance,TaxiIn,TaxiOut,ArrDelay_delayed
id,,,,,,,
1787870,134.0,96.0,89.0,403,14.0,24.0,1
2264296,336.0,310.0,8.0,2342,10.0,16.0,1
3029928,62.0,43.0,31.0,223,4.0,15.0,1
6028125,214.0,199.0,13.0,1390,6.0,9.0,1
3318600,75.0,45.0,30.0,171,3.0,27.0,1
...,...,...,...,...,...,...,...
5243010,173.0,152.0,21.0,1123,7.0,14.0,1
3690122,244.0,234.0,59.0,1999,4.0,6.0,1
1993642,56.0,35.0,92.0,177,8.0,13.0,1


,ActualElapsedTime,AirTime,DepDelay,Distance,TaxiIn,TaxiOut
id,,,,,,
1787870,134.0,96.0,89.0,403,14.0,24.0
2264296,336.0,310.0,8.0,2342,10.0,16.0
3029928,62.0,43.0,31.0,223,4.0,15.0
6028125,214.0,199.0,13.0,1390,6.0,9.0
3318600,75.0,45.0,30.0,171,3.0,27.0


id
1787870    1
2264296    1
3029928    1
6028125    1
3318600    1
Name: ArrDelay_delayed, dtype: int32

In [30]:
# Model de xarxes neuronals
xarxa_neuronal = MLPClassifier(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=500, random_state=5678)
xarxa_neuronal.fit(mostra_entrenament_x, mostra_entrenament_y)

# Generem la predicció
xarxa_neuronal_prediccio = xarxa_neuronal.predict(mostra_validacio_x)

In [31]:
# Model logístic
logistica = LogisticRegression(max_iter=1000, random_state=5678)
logistica.fit(mostra_entrenament_x, mostra_entrenament_y)

# Generem la predicció
logistica_prediccio = logistica.predict(mostra_validacio_x)

In [32]:
# Model XGBoost
xgboost = xgb.XGBClassifier(objective ='binary:logistic', use_label_encoder=False, random_state=5678)
xgboost.fit(mostra_entrenament_x, mostra_entrenament_y)

# Generem la predicció
xgboost_prediccio = xgboost.predict(mostra_validacio_x)

[12:23:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


- Exercici 2

    Compara els models de classificació utilitzant la precisió (accuracy), una matriu de confiança i d’altres mètriques més avançades.

In [33]:
# Calculem el grau de exactitud

# Model de xarxes neuronals
xarxa_neuronal_exactitud = round(metrics.accuracy_score(mostra_validacio_y, xarxa_neuronal_prediccio),4)
print(xarxa_neuronal_exactitud)

# Model logístic
logistica_exactitud = round(metrics.accuracy_score(mostra_validacio_y, logistica_prediccio),4)
print(logistica_exactitud)

# Model XGBoost
xgboost_exactitud = round(metrics.accuracy_score(mostra_validacio_y, xgboost_prediccio),4)
print(xgboost_exactitud)

0.8973
0.901
0.8977


El model més exacte és el de XGBoost

In [34]:
# Calculem la matriu de confusió

# Model de xarxes neuronals
xarxa_neuronal_matriu_confusio = metrics.confusion_matrix(mostra_validacio_y, xarxa_neuronal_prediccio)
print(xarxa_neuronal_matriu_confusio)

# Model logístic
logistica_matriu_confusio = metrics.confusion_matrix(mostra_validacio_y, logistica_prediccio)
print(logistica_matriu_confusio)

# Model XGBoost
xgboost_matriu_confusio = metrics.confusion_matrix(mostra_validacio_y, xgboost_prediccio)
print(xgboost_matriu_confusio)

[[  80  250]
 [  58 2612]]
[[  81  249]
 [  48 2622]]
[[ 117  213]
 [  94 2576]]


El model amb una matriu de confusió amb més verdaders positius i verdaders negatius és el de XGBoost.

In [35]:
# Calculem el grau de precisió

# Model de xarxes neuronals
xarxa_neuronal_precisio = round(metrics.precision_score(mostra_validacio_y, xarxa_neuronal_prediccio),4)
print(xarxa_neuronal_precisio)

# Model logístic
logistica_precisio = round(metrics.precision_score(mostra_validacio_y, logistica_prediccio),4)
print(logistica_precisio)

# Model XGBoost
xgboost_precisio = round(metrics.precision_score(mostra_validacio_y, xgboost_prediccio),4)
print(xgboost_precisio)

0.9126
0.9133
0.9236


El model més precís és el XGBoost

In [36]:
# Calculem l'àrea sota la corba

# Model de xarxes neuronals
xarxa_neuronal_asc = round(metrics.roc_auc_score(mostra_validacio_y, xarxa_neuronal_prediccio),4)
print(xarxa_neuronal_asc)

# Model logístic
logistica_asc = round(metrics.roc_auc_score(mostra_validacio_y, logistica_prediccio),4)
print(logistica_asc)

# Model XGBoost
xgboost_asc = round(metrics.roc_auc_score(mostra_validacio_y, xgboost_prediccio),4)
print(xgboost_asc)

0.6104
0.6137
0.6597


El model amb una àrea sota la corba és el XGBoost

- Exercici 3

    Entrena’ls utilitzant els diferents paràmetres que admeten.

In [37]:
# Model xarxa neuronal

# Definició dels paràmetres a afinar
xarxa_neuronal_parametres_a_afinar = {'hidden_layer_sizes': [[8,8],[8,8,8],[8,8,8,8]], 'activation': ['identity', 'logistic', 'tanh', 'relu'],'solver': ['lbfgs', 'sgd', 'adam'],}

# Métrica escollida d'entre: precision, recall, f1, f1_weighted, f1_micro, f1_macro, accuracy, roc_auc
xarxa_neuronal_metrica = 'accuracy'

# Entrenament dels hiper_paràmetres de la métrica escollida
xarxa_neuronal_entrenament = GridSearchCV(estimator = MLPClassifier(max_iter=500,random_state=5678), param_grid = xarxa_neuronal_parametres_a_afinar, cv=10, scoring='%s' % xarxa_neuronal_metrica, n_jobs= -1)
# Ajust
xarxa_neuronal_entrenament.fit(mostra_entrenament_x, mostra_entrenament_y)
    
print("Els paràmetres que afinen millor la métrica són: \n{}\n".format(xarxa_neuronal_entrenament.best_params_))

xarxa_neuronal_metrica_mitjana = xarxa_neuronal_entrenament.cv_results_['mean_test_score']
xarxa_neuronal_metrica_desviacio_estandar = xarxa_neuronal_entrenament.cv_results_['std_test_score']

Els paràmetres que afinen millor la métrica són: 
{'activation': 'relu', 'hidden_layer_sizes': [8, 8, 8, 8], 'solver': 'lbfgs'}



In [38]:
print("Llista de puntuacións obtinguda en les diferents combinacions de hiper_paràmetres:\n")
print("Puntuació   Desv. Estd.     Paràmetres")
print("====================================================================================================")

for mitjana, desviacio_estandard, parametres in zip(np.sort(xarxa_neuronal_metrica_mitjana)[::-1], xarxa_neuronal_metrica_desviacio_estandar, xarxa_neuronal_entrenament.cv_results_['params']):
    print("{}   |  [+/-{}]  |  {}".format(round(mitjana,4), round(desviacio_estandard,4), parametres))
    print("____________________________________________________________________________________________________")

Llista de puntuacións obtinguda en les diferents combinacions de hiper_paràmetres:

Puntuació   Desv. Estd.     Paràmetres
0.9128   |  [+/-0.0]  |  {'activation': 'identity', 'hidden_layer_sizes': [8, 8], 'solver': 'lbfgs'}
____________________________________________________________________________________________________
0.912   |  [+/-0.0]  |  {'activation': 'identity', 'hidden_layer_sizes': [8, 8], 'solver': 'sgd'}
____________________________________________________________________________________________________
0.9119   |  [+/-0.0204]  |  {'activation': 'identity', 'hidden_layer_sizes': [8, 8], 'solver': 'adam'}
____________________________________________________________________________________________________
0.9118   |  [+/-0.0003]  |  {'activation': 'identity', 'hidden_layer_sizes': [8, 8, 8], 'solver': 'lbfgs'}
____________________________________________________________________________________________________
0.9117   |  [+/-0.0]  |  {'activation': 'identity', 'hidden_laye

In [39]:
# Model logístic

# Definició dels paràmetres a afinar
logistica_parametres_a_afinar = {'penalty': ['l2', 'none'],'C': [0.5,1.0,1.5],'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],}

# Métrica escollida d'entre: precision, recall, f1, f1_weighted, f1_micro, f1_macro, accuracy, roc_auc
logistica_metrica = 'accuracy'

# Entrenament dels hiper_paràmetres de la métrica escollida
logistica_entrenament = GridSearchCV(estimator = LogisticRegression(max_iter = 1000), param_grid = logistica_parametres_a_afinar, cv=10, scoring='%s' % logistica_metrica, n_jobs= -1)
# Ajust
logistica_entrenament.fit(mostra_entrenament_x, mostra_entrenament_y)
    
print("Els paràmetres que afinen millor la métrica són: \n{}\n".format(logistica_entrenament.best_params_))

logistica_metrica_mitjana = logistica_entrenament.cv_results_['mean_test_score']
logistica_metrica_desviacio_estandar = logistica_entrenament.cv_results_['std_test_score']

Els paràmetres que afinen millor la métrica són: 
{'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}



In [40]:
print("Llista de puntuacions obtinguda en les diferents combinacions de hiper_paràmetres:\n")
print("Puntuació   Desv. Estd.     Paràmetres")
print("====================================================================================================")

for mitjana, desviacio_estandard, parametres in zip(np.sort(logistica_metrica_mitjana)[::-1], logistica_metrica_desviacio_estandar, logistica_entrenament.cv_results_['params']):
    print("{}   |  [+/-{}]  |  {}".format(round(mitjana,4), round(desviacio_estandard,4), parametres))
    print("____________________________________________________________________________________________________")

Llista de puntuacions obtinguda en les diferents combinacions de hiper_paràmetres:

Puntuació   Desv. Estd.     Paràmetres
0.9126   |  [+/-0.0048]  |  {'C': 0.5, 'penalty': 'l2', 'solver': 'newton-cg'}
____________________________________________________________________________________________________
0.9125   |  [+/-0.0048]  |  {'C': 0.5, 'penalty': 'l2', 'solver': 'lbfgs'}
____________________________________________________________________________________________________
0.9125   |  [+/-0.0039]  |  {'C': 0.5, 'penalty': 'l2', 'solver': 'sag'}
____________________________________________________________________________________________________
0.9125   |  [+/-0.0038]  |  {'C': 0.5, 'penalty': 'l2', 'solver': 'saga'}
____________________________________________________________________________________________________
0.9125   |  [+/-0.0048]  |  {'C': 0.5, 'penalty': 'none', 'solver': 'newton-cg'}
___________________________________________________________________________________________

In [41]:
# Model XGBoost

# Definició dels paràmetres a afinar
xgboost_parametres_a_afinar = {'booster': ['gbtree', 'gblinear', 'dart'],}

# Métrica escollida d'entre: precision, recall, f1, f1_weighted, f1_micro, f1_macro, accuracy, roc_auc
xgboost_metrica = 'accuracy'

# Entrenament dels hiper_paràmetres de la métrica escollida
xgboost_entrenament = GridSearchCV(estimator = xgb.XGBClassifier(objective ='binary:logistic', use_label_encoder = False, random_state = 5678), param_grid = xgboost_parametres_a_afinar, cv=10, scoring='%s' % xgboost_metrica, n_jobs= -1)
# Ajust
xgboost_entrenament.fit(mostra_entrenament_x, mostra_entrenament_y)
    
print("Els paràmetres que afinen millor la métrica són: \n{}\n".format(xgboost_entrenament.best_params_))

xgboost_metrica_mitjana = xgboost_entrenament.cv_results_['mean_test_score']
xgboost_metrica_desviacio_estandar = xgboost_entrenament.cv_results_['std_test_score']

[12:37:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Els paràmetres que afinen millor la métrica són: 
{'booster': 'gbtree'}



In [42]:
print("Llista de puntuacions obtinguda en les diferents combinacions de hiper_paràmetres:\n")
print("Puntuació   Desv. Estd.     Paràmetres")
print("====================================================================================================")

for mitjana, desviacio_estandard, parametres in zip(np.sort(xgboost_metrica_mitjana)[::-1], xgboost_metrica_desviacio_estandar, xgboost_entrenament.cv_results_['params']):
    print("{}   |  [+/-{}]  |  {}".format(round(mitjana,4), round(desviacio_estandard,4), parametres))
    print("____________________________________________________________________________________________________")

Llista de puntuacions obtinguda en les diferents combinacions de hiper_paràmetres:

Puntuació   Desv. Estd.     Paràmetres
0.9109   |  [+/-0.0023]  |  {'booster': 'gbtree'}
____________________________________________________________________________________________________
0.9109   |  [+/-0.0047]  |  {'booster': 'gblinear'}
____________________________________________________________________________________________________
0.9095   |  [+/-0.0023]  |  {'booster': 'dart'}
____________________________________________________________________________________________________


- Exercici 4

    Compara el seu rendiment utilitzant l’aproximació traint/test o cross-validation.

In [43]:
# Model xarxa neuronal

xarxa_neuronal_prediccio_rendiment = xarxa_neuronal.predict(mostra_validacio_x)
xarxa_neuronal_metrica_exactitud_prediccio_rendiment = round(metrics.accuracy_score(mostra_validacio_y, xarxa_neuronal_prediccio_rendiment),4)
xarxa_neuronal_metrica_asc_prediccio_rendiment = round(metrics.roc_auc_score(mostra_validacio_y, xarxa_neuronal_prediccio_rendiment),4)

#print(classification_report(mostra_validacio_y, xarxa_neuronal_prediccio_rendiment))
print("Exactitud:           ", xarxa_neuronal_metrica_exactitud_prediccio_rendiment)
print("Àrea sota la corba  :", xarxa_neuronal_metrica_asc_prediccio_rendiment)

Exactitud:            0.8973
Àrea sota la corba  : 0.6104


In [44]:
# Model logístic

logistica_prediccio_rendiment = logistica.predict(mostra_validacio_x)
logistica_metrica_exactitud_prediccio_rendiment = round(metrics.accuracy_score(mostra_validacio_y, logistica_prediccio_rendiment),4)
logistica_metrica_asc_prediccio_rendiment = round(metrics.roc_auc_score(mostra_validacio_y, logistica_prediccio_rendiment),4)

#print(classification_report(mostra_validacio_y, logistica_prediccio_rendiment))
print("Exactitud:           ", logistica_metrica_exactitud_prediccio_rendiment)
print("Àrea sota la corba  :", logistica_metrica_asc_prediccio_rendiment)

Exactitud:            0.901
Àrea sota la corba  : 0.6137


In [45]:
# Model XGBoost

xgboost_prediccio_rendiment = xgboost.predict(mostra_validacio_x)
xgboost_metrica_exactitud_prediccio_rendiment = round(metrics.accuracy_score(mostra_validacio_y, xgboost_prediccio_rendiment),4)
xgboost_metrica_asc_prediccio_rendiment = round(metrics.roc_auc_score(mostra_validacio_y, xgboost_prediccio_rendiment),4)

#print(classification_report(mostra_validacio_y, xgboost_prediccio_rendiment))
print("Exactitud:           ", xgboost_metrica_exactitud_prediccio_rendiment)
print("Àrea sota la corba:  ", xgboost_metrica_asc_prediccio_rendiment)

Exactitud:            0.8977
Àrea sota la corba:   0.6597


## Nivell 2

- Exercici 5

    Realitza algun procés d’enginyeria de variables per millorar-ne la predicció.

In [46]:
# Refarem el dataframe de vols per tal de normalitzar-lo
# Primer mirem quines columnes hi ha a normalitzar, és a dir, numériques
columnes = []
columnes.extend(x.columns)
columnes.insert(len(columnes),y.name)
columnes_a_normalitzar = []
columnes_a_normalitzar.extend(x.columns)

print(columnes_a_normalitzar)

['ActualElapsedTime', 'AirTime', 'DepDelay', 'Distance', 'TaxiIn', 'TaxiOut']


In [47]:
# Fem cópia de 'vols'
vols_normalitzat = vols.copy()
vols_normalitzat = vols_normalitzat[columnes]

# Normalitzem 'vols' --> 'vols_normalitzat'
normalitzador = preprocessing.MinMaxScaler()
vols_normalitzat_normalitzades = normalitzador.fit_transform(vols_normalitzat[columnes_a_normalitzar])
display(vols_normalitzat_normalitzades)

array([[0.18646617, 0.19543974, 0.01464844, 0.18373555, 0.0625    ,
        0.0224359 ],
       [0.15789474, 0.15798046, 0.03808594, 0.13100791, 0.05208333,
        0.03846154],
       [0.09774436, 0.07166124, 0.00390625, 0.05252484, 0.03125   ,
        0.08653846],
       ...,
       [0.21052632, 0.20358306, 0.06835938, 0.18130197, 0.05208333,
        0.06089744],
       [0.16691729, 0.12540717, 0.01269531, 0.09815453, 0.27083333,
        0.05448718],
       [0.06616541, 0.04560261, 0.        , 0.03346177, 0.13541667,
        0.03846154]])

In [48]:
# Materialitzem la normalització en el dataframe
k = 0
print(columnes_a_normalitzar)

for columna in columnes_a_normalitzar:
    vols_normalitzat[columna] = vols_normalitzat_normalitzades[:,k]
    k = k + 1

display(vols_normalitzat)

['ActualElapsedTime', 'AirTime', 'DepDelay', 'Distance', 'TaxiIn', 'TaxiOut']


,ActualElapsedTime,AirTime,DepDelay,Distance,TaxiIn,TaxiOut,ArrDelay_delayed
id,,,,,,,
3696449,0.186466,0.195440,0.014648,0.183736,0.062500,0.022436,1
6652612,0.157895,0.157980,0.038086,0.131008,0.052083,0.038462,1
3639563,0.097744,0.071661,0.003906,0.052525,0.031250,0.086538,0
2194623,0.088722,0.042345,0.002930,0.029609,0.052083,0.118590,1
6222288,0.524812,0.480456,0.007812,0.495640,0.395833,0.080128,1
...,...,...,...,...,...,...,...
2713469,0.069173,0.063518,0.022461,0.028392,0.083333,0.025641,1
5926114,0.201504,0.192182,0.166016,0.211925,0.125000,0.041667,1
570932,0.210526,0.203583,0.068359,0.181302,0.052083,0.060897,1


In [49]:
## Variable dependent
y = vols_normalitzat['ArrDelay_delayed']

# Variables predictores
x = vols_normalitzat.drop(['ArrDelay_delayed'], axis = 1)

# LLista amb el nom dels predictors
x_columnes = list(x.columns)

In [50]:
# Creació de la mostra d'entrenament i validació
mostra_entrenament_normalitzada_x, mostra_validacio_normalitzada_x, mostra_entrenament_normalitzada_y, mostra_validacio_normalitzada_y = train_test_split(x, y, test_size = 0.2, random_state=567)

mostra_entrenament_normalitzada = pd.concat([mostra_entrenament_normalitzada_x, mostra_entrenament_normalitzada_y], axis=1)
mostra_validacio_normalitzada = pd.concat([mostra_validacio_normalitzada_x, mostra_validacio_normalitzada_y], axis=1)

print("Tamany de mostra d'entrenament: {}".format(mostra_entrenament_normalitzada_x.shape[0]))
print("Tamany de mostra de validació: {}".format(mostra_validacio_normalitzada_x.shape[0]))

display(mostra_entrenament_normalitzada)
display(mostra_entrenament_normalitzada_x.head())
display(mostra_entrenament_normalitzada_y.head())

display(mostra_validacio_normalitzada)
display(mostra_validacio_normalitzada_x.head())
display(mostra_validacio_normalitzada_y.head())

Tamany de mostra d'entrenament: 12000
Tamany de mostra de validació: 3000


,ActualElapsedTime,AirTime,DepDelay,Distance,TaxiIn,TaxiOut,ArrDelay_delayed
id,,,,,,,
1020517,0.171429,0.154723,0.084961,0.132225,0.072917,0.067308,1
751521,0.135338,0.122150,0.003906,0.079903,0.145833,0.032051,0
1528665,0.142857,0.141694,0.014648,0.115798,0.041667,0.041667,1
815631,0.445113,0.462541,0.024414,0.542081,0.062500,0.048077,1
1169295,0.557895,0.516287,0.012695,0.449605,0.062500,0.182692,1
...,...,...,...,...,...,...,...
3365651,0.482707,0.490228,0.013672,0.466031,0.114583,0.057692,1
1214504,0.114286,0.112378,0.017578,0.084364,0.031250,0.041667,1
886636,0.072180,0.055375,0.008789,0.048672,0.052083,0.057692,1


,ActualElapsedTime,AirTime,DepDelay,Distance,TaxiIn,TaxiOut
id,,,,,,
1020517,0.171429,0.154723,0.084961,0.132225,0.072917,0.067308
751521,0.135338,0.122150,0.003906,0.079903,0.145833,0.032051
1528665,0.142857,0.141694,0.014648,0.115798,0.041667,0.041667
815631,0.445113,0.462541,0.024414,0.542081,0.062500,0.048077
1169295,0.557895,0.516287,0.012695,0.449605,0.062500,0.182692


id
1020517    1
751521     0
1528665    1
815631     1
1169295    1
Name: ArrDelay_delayed, dtype: int32

,ActualElapsedTime,AirTime,DepDelay,Distance,TaxiIn,TaxiOut,ArrDelay_delayed
id,,,,,,,
689285,0.357895,0.351792,0.009766,0.404786,0.052083,0.083333,1
1513378,0.066165,0.042345,0.000977,0.035895,0.052083,0.070513,0
80624,0.114286,0.105863,0.069336,0.074630,0.052083,0.048077,1
1239004,0.064662,0.066775,0.002930,0.053133,0.041667,0.022436,1
1327794,0.066165,0.052117,0.008789,0.018252,0.062500,0.048077,1
...,...,...,...,...,...,...,...
6618180,0.064662,0.030945,0.083984,0.013182,0.041667,0.092949,1
5023448,0.055639,0.045603,0.001953,0.024133,0.052083,0.041667,1
649241,0.187970,0.193811,0.036133,0.152099,0.062500,0.028846,1


,ActualElapsedTime,AirTime,DepDelay,Distance,TaxiIn,TaxiOut
id,,,,,,
689285,0.357895,0.351792,0.009766,0.404786,0.052083,0.083333
1513378,0.066165,0.042345,0.000977,0.035895,0.052083,0.070513
80624,0.114286,0.105863,0.069336,0.074630,0.052083,0.048077
1239004,0.064662,0.066775,0.002930,0.053133,0.041667,0.022436
1327794,0.066165,0.052117,0.008789,0.018252,0.062500,0.048077


id
689285     1
1513378    0
80624      1
1239004    1
1327794    1
Name: ArrDelay_delayed, dtype: int32

In [51]:
# Model de xarxes neuronals
xarxa_neuronal_normalitzada = MLPClassifier(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=500, random_state=5678)
xarxa_neuronal_normalitzada.fit(mostra_entrenament_normalitzada_x, mostra_entrenament_normalitzada_y)

# Generem la predicció
xarxa_neuronal_normalitzada_prediccio = xarxa_neuronal_normalitzada.predict(mostra_validacio_normalitzada_x)

In [52]:
# Model logístic
logistica_normalitzada = LogisticRegression(max_iter=1000, random_state=5678)
logistica_normalitzada.fit(mostra_entrenament_normalitzada_x, mostra_entrenament_normalitzada_y)

# Generem la predicció
logistica_normalitzada_prediccio = logistica_normalitzada.predict(mostra_validacio_normalitzada_x)

In [53]:
# Model xgboost_normalitzada
xgboost_normalitzada = xgb.XGBClassifier(objective ='binary:logistic', use_label_encoder=False, random_state=5678)
xgboost_normalitzada.fit(mostra_entrenament_normalitzada_x, mostra_entrenament_normalitzada_y)

# Generem la predicció
xgboost_normalitzada_prediccio = xgboost_normalitzada.predict(mostra_validacio_normalitzada_x)

[12:37:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Comparativa de models de classificació utilitzant la precisió (accuracy), una matriu de confiança i d’altres mètriques més avançades

In [54]:
# Calculem el grau de exactitud

# Model de xarxes neuronals
xarxa_neuronal_normalitzada_exactitud = round(metrics.accuracy_score(mostra_validacio_normalitzada_y, xarxa_neuronal_normalitzada_prediccio),4)
print(xarxa_neuronal_normalitzada_exactitud)

# Model logístic
logistica_normalitzada_exactitud = round(metrics.accuracy_score(mostra_validacio_normalitzada_y, logistica_normalitzada_prediccio),4)
print(logistica_normalitzada_exactitud)

# Model xgboost_normalitzada
xgboost_normalitzada_exactitud = round(metrics.accuracy_score(mostra_validacio_normalitzada_y, xgboost_normalitzada_prediccio),4)
print(xgboost_normalitzada_exactitud)

0.9203
0.9037
0.9117


El model més exacte és el de xarxes neuronals

In [55]:
# Calculem la matriu de confusió

# Model de xarxes neuronals
xarxa_neuronal_normalitzada_matriu_confusio = metrics.confusion_matrix(mostra_validacio_normalitzada_y, xarxa_neuronal_normalitzada_prediccio)
print(xarxa_neuronal_normalitzada_matriu_confusio)

# Model logístic
logistica_normalitzada_matriu_confusio = metrics.confusion_matrix(mostra_validacio_normalitzada_y, logistica_normalitzada_prediccio)
print(logistica_normalitzada_matriu_confusio)

# Model xgboost_normalitzada
xgboost_normalitzada_matriu_confusio = metrics.confusion_matrix(mostra_validacio_normalitzada_y, xgboost_normalitzada_prediccio)
print(xgboost_normalitzada_matriu_confusio)

[[  97  192]
 [  47 2664]]
[[   0  289]
 [   0 2711]]
[[ 130  159]
 [ 106 2605]]


El model amb una matriu de confusió amb més verdaders positius i verdaders negatius és el logístic.


In [56]:
# Calculem el grau de precisió

# Model de xarxes neuronals
xarxa_neuronal_normalitzada_precisio = round(metrics.precision_score(mostra_validacio_normalitzada_y, xarxa_neuronal_normalitzada_prediccio),4)
print(xarxa_neuronal_normalitzada_precisio)

# Model logístic
logistica_normalitzada_precisio = round(metrics.precision_score(mostra_validacio_normalitzada_y, logistica_normalitzada_prediccio),4)
print(logistica_normalitzada_precisio)

# Model xgboost_normalitzada
xgboost_normalitzada_precisio = round(metrics.precision_score(mostra_validacio_normalitzada_y, xgboost_normalitzada_prediccio),4)
print(xgboost_normalitzada_precisio)

0.9328
0.9037
0.9425


El model més precís és el xgboost

In [57]:
# Calculem l'àrea sota la corba

# Model de xarxes neuronals
xarxa_neuronal_normalitzada_asc = round(metrics.roc_auc_score(mostra_validacio_normalitzada_y, xarxa_neuronal_normalitzada_prediccio),4)
print(xarxa_neuronal_normalitzada_asc)

# Model logístic
logistica_normalitzada_asc = round(metrics.roc_auc_score(mostra_validacio_normalitzada_y, logistica_normalitzada_prediccio),4)
print(logistica_normalitzada_asc)

# Model xgboost_normalitzada
xgboost_normalitzada_asc = round(metrics.roc_auc_score(mostra_validacio_normalitzada_y, xgboost_normalitzada_prediccio),4)
print(xgboost_normalitzada_asc)

0.6592
0.5
0.7054


El model amb una àrea sota la corba més alt és el xgboost

Faig entrenament utilitzant els diferents paràmetres que admeten.

In [58]:
# Model xarxa neuronal

# Definició dels paràmetres a afinar
xarxa_neuronal_normalitzada_parametres_a_afinar = {'hidden_layer_sizes': [[8,8],[8,8,8],[8,8,8,8]], 'activation': ['identity', 'logistic', 'tanh', 'relu'],'solver': ['lbfgs', 'sgd', 'adam'],}

# Métrica escollida d'entre: precision, recall, f1, f1_weighted, f1_micro, f1_macro, accuracy, roc_auc
xarxa_neuronal_normalitzada_metrica = 'accuracy'

# Entrenament dels hiper_paràmetres de la métrica escollida
xarxa_neuronal_normalitzada_entrenament = GridSearchCV(estimator = MLPClassifier(max_iter=500,random_state=5678), param_grid = xarxa_neuronal_normalitzada_parametres_a_afinar, cv=10, scoring='%s' % xarxa_neuronal_normalitzada_metrica, n_jobs= -1)
# Ajust
xarxa_neuronal_normalitzada_entrenament.fit(mostra_entrenament_normalitzada_x, mostra_entrenament_normalitzada_y)
    
print("Els paràmetres que afinen millor la métrica són: \n{}\n".format(xarxa_neuronal_normalitzada_entrenament.best_params_))

xarxa_neuronal_normalitzada_metrica_mitjana = xarxa_neuronal_normalitzada_entrenament.cv_results_['mean_test_score']
xarxa_neuronal_normalitzada_metrica_desviacio_estandar = xarxa_neuronal_normalitzada_entrenament.cv_results_['std_test_score']

Els paràmetres que afinen millor la métrica són: 
{'activation': 'relu', 'hidden_layer_sizes': [8, 8, 8], 'solver': 'lbfgs'}



C:\Users\Antoni\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [59]:
print("Llista de puntuacións obtinguda en les diferents combinacions de hiper_paràmetres:\n")
print("Puntuació   Desv. Estd.     Paràmetres")
print("====================================================================================================")

for mitjana, desviacio_estandard, parametres in zip(np.sort(xarxa_neuronal_normalitzada_metrica_mitjana)[::-1], xarxa_neuronal_normalitzada_metrica_desviacio_estandar, xarxa_neuronal_normalitzada_entrenament.cv_results_['params']):
    print("{}   |  [+/-{}]  |  {}".format(round(mitjana,4), round(desviacio_estandard,4), parametres))
    print("____________________________________________________________________________________________________")

Llista de puntuacións obtinguda en les diferents combinacions de hiper_paràmetres:

Puntuació   Desv. Estd.     Paràmetres
0.9121   |  [+/-0.007]  |  {'activation': 'identity', 'hidden_layer_sizes': [8, 8], 'solver': 'lbfgs'}
____________________________________________________________________________________________________
0.9112   |  [+/-0.0027]  |  {'activation': 'identity', 'hidden_layer_sizes': [8, 8], 'solver': 'sgd'}
____________________________________________________________________________________________________
0.9109   |  [+/-0.0077]  |  {'activation': 'identity', 'hidden_layer_sizes': [8, 8], 'solver': 'adam'}
____________________________________________________________________________________________________
0.9108   |  [+/-0.007]  |  {'activation': 'identity', 'hidden_layer_sizes': [8, 8, 8], 'solver': 'lbfgs'}
____________________________________________________________________________________________________
0.9107   |  [+/-0.0063]  |  {'activation': 'identity', 'hid

In [60]:
# Model logístic

# Definició dels paràmetres a afinar
logistica_normalitzada_parametres_a_afinar = {'penalty': ['l2', 'none'],'C': [0.5,1.0,1.5],'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],}

# Métrica escollida d'entre: precision, recall, f1, f1_weighted, f1_micro, f1_macro, accuracy, roc_auc
logistica_normalitzada_metrica = 'accuracy'

# Entrenament dels hiper_paràmetres de la métrica escollida
logistica_normalitzada_entrenament = GridSearchCV(estimator = LogisticRegression(max_iter = 1000), param_grid = logistica_normalitzada_parametres_a_afinar, cv=10, scoring='%s' % logistica_normalitzada_metrica, n_jobs= -1)
# Ajust
logistica_normalitzada_entrenament.fit(mostra_entrenament_normalitzada_x, mostra_entrenament_normalitzada_y)
    
print("Els paràmetres que afinen millor la métrica són: \n{}\n".format(logistica_normalitzada_entrenament.best_params_))

logistica_normalitzada_metrica_mitjana = logistica_normalitzada_entrenament.cv_results_['mean_test_score']
logistica_normalitzada_metrica_desviacio_estandar = logistica_normalitzada_entrenament.cv_results_['std_test_score']

C:\Users\Antoni\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


Els paràmetres que afinen millor la métrica són: 
{'C': 0.5, 'penalty': 'none', 'solver': 'saga'}



In [61]:
print("Llista de puntuacions obtinguda en les diferents combinacions de hiper_paràmetres:\n")
print("Puntuació   Desv. Estd.     Paràmetres")
print("====================================================================================================")

for mitjana, desviacio_estandard, parametres in zip(np.sort(logistica_normalitzada_metrica_mitjana)[::-1], logistica_normalitzada_metrica_desviacio_estandar, logistica_normalitzada_entrenament.cv_results_['params']):
    print("{}   |  [+/-{}]  |  {}".format(round(mitjana,4), round(desviacio_estandard,4), parametres))
    print("____________________________________________________________________________________________________")

Llista de puntuacions obtinguda en les diferents combinacions de hiper_paràmetres:

Puntuació   Desv. Estd.     Paràmetres
0.9081   |  [+/-0.0006]  |  {'C': 0.5, 'penalty': 'l2', 'solver': 'newton-cg'}
____________________________________________________________________________________________________
0.9081   |  [+/-0.0006]  |  {'C': 0.5, 'penalty': 'l2', 'solver': 'lbfgs'}
____________________________________________________________________________________________________
0.9081   |  [+/-0.0006]  |  {'C': 0.5, 'penalty': 'l2', 'solver': 'sag'}
____________________________________________________________________________________________________
0.908   |  [+/-0.0006]  |  {'C': 0.5, 'penalty': 'l2', 'solver': 'saga'}
____________________________________________________________________________________________________
0.908   |  [+/-0.007]  |  {'C': 0.5, 'penalty': 'none', 'solver': 'newton-cg'}
______________________________________________________________________________________________

In [62]:
# Model xgboost_normalitzada

# Definició dels paràmetres a afinar
xgboost_normalitzada_parametres_a_afinar = {'booster': ['gbtree', 'gblinear', 'dart'],}

# Métrica escollida d'entre: precision, recall, f1, f1_weighted, f1_micro, f1_macro, accuracy, roc_auc
xgboost_normalitzada_metrica = 'accuracy'

# Entrenament dels hiper_paràmetres de la métrica escollida
xgboost_normalitzada_entrenament = GridSearchCV(estimator = xgb.XGBClassifier(objective ='binary:logistic', use_label_encoder = False, random_state = 5678), param_grid = xgboost_normalitzada_parametres_a_afinar, cv=10, scoring='%s' % xgboost_normalitzada_metrica, n_jobs= -1)
# Ajust
xgboost_normalitzada_entrenament.fit(mostra_entrenament_normalitzada_x, mostra_entrenament_normalitzada_y)
    
print("Els paràmetres que afinen millor la métrica són: \n{}\n".format(xgboost_normalitzada_entrenament.best_params_))

xgboost_normalitzada_metrica_mitjana = xgboost_normalitzada_entrenament.cv_results_['mean_test_score']
xgboost_normalitzada_metrica_desviacio_estandar = xgboost_normalitzada_entrenament.cv_results_['std_test_score']

[12:52:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Els paràmetres que afinen millor la métrica són: 
{'booster': 'gblinear'}



In [63]:
print("Llista de puntuacions obtinguda en les diferents combinacions de hiper_paràmetres:\n")
print("Puntuació   Desv. Estd.     Paràmetres")
print("====================================================================================================")

for mitjana, desviacio_estandard, parametres in zip(np.sort(xgboost_normalitzada_metrica_mitjana)[::-1], xgboost_normalitzada_metrica_desviacio_estandar, xgboost_normalitzada_entrenament.cv_results_['params']):
    print("{}   |  [+/-{}]  |  {}".format(round(mitjana,4), round(desviacio_estandard,4), parametres))
    print("____________________________________________________________________________________________________")

Llista de puntuacions obtinguda en les diferents combinacions de hiper_paràmetres:

Puntuació   Desv. Estd.     Paràmetres
0.9064   |  [+/-0.008]  |  {'booster': 'gbtree'}
____________________________________________________________________________________________________
0.9037   |  [+/-0.0047]  |  {'booster': 'gblinear'}
____________________________________________________________________________________________________
0.9037   |  [+/-0.008]  |  {'booster': 'dart'}
____________________________________________________________________________________________________


Comparo el seu rendiment utilitzant l’aproximació traint/test o cross-validation.

In [64]:
# Model xarxa neuronal

xarxa_neuronal_normalitzada_prediccio_rendiment = xarxa_neuronal_normalitzada.predict(mostra_validacio_normalitzada_x)
xarxa_neuronal_normalitzada_metrica_exactitud_prediccio_rendiment = round(metrics.accuracy_score(mostra_validacio_normalitzada_y, xarxa_neuronal_normalitzada_prediccio_rendiment),4)
xarxa_neuronal_normalitzada_metrica_asc_prediccio_rendiment = round(metrics.roc_auc_score(mostra_validacio_normalitzada_y, xarxa_neuronal_normalitzada_prediccio_rendiment),4)

#print(classification_report(mostra_validacio_normalitzada_y, xarxa_neuronal_normalitzada_prediccio_rendiment))
print("Exactitud:           ", xarxa_neuronal_normalitzada_metrica_exactitud_prediccio_rendiment)
print("Àrea sota la corba  :", xarxa_neuronal_normalitzada_metrica_asc_prediccio_rendiment)

Exactitud:            0.9203
Àrea sota la corba  : 0.6592


In [65]:
# Model logístic

logistica_normalitzada_prediccio_rendiment = logistica_normalitzada.predict(mostra_validacio_normalitzada_x)
logistica_normalitzada_metrica_exactitud_prediccio_rendiment = round(metrics.accuracy_score(mostra_validacio_normalitzada_y, logistica_normalitzada_prediccio_rendiment),4)
logistica_normalitzada_metrica_asc_prediccio_rendiment = round(metrics.roc_auc_score(mostra_validacio_normalitzada_y, logistica_normalitzada_prediccio_rendiment),4)

#print(classification_report(mostra_validacio_normalitzada_y, logistica_normalitzada_prediccio_rendiment))
print("Exactitud:           ", logistica_normalitzada_metrica_exactitud_prediccio_rendiment)
print("Àrea sota la corba  :", logistica_normalitzada_metrica_asc_prediccio_rendiment)

Exactitud:            0.9037
Àrea sota la corba  : 0.5


In [66]:
# Model xgboost_normalitzada

xgboost_normalitzada_prediccio_rendiment = xgboost_normalitzada.predict(mostra_validacio_normalitzada_x)
xgboost_normalitzada_metrica_exactitud_prediccio_rendiment = round(metrics.accuracy_score(mostra_validacio_normalitzada_y, xgboost_normalitzada_prediccio_rendiment),4)
xgboost_normalitzada_metrica_asc_prediccio_rendiment = round(metrics.roc_auc_score(mostra_validacio_normalitzada_y, xgboost_normalitzada_prediccio_rendiment),4)

#print(classification_report(mostra_validacio_normalitzada_y, xgboost_normalitzada_prediccio_rendiment))
print("Exactitud:           ", xgboost_normalitzada_metrica_exactitud_prediccio_rendiment)
print("Àrea sota la corba:  ", xgboost_normalitzada_metrica_asc_prediccio_rendiment)

Exactitud:            0.9117
Àrea sota la corba:   0.7054


## Nivell 3

- Exercici 6

    No utilitzis la variable DepDelay a l’hora de fer prediccions.

In [67]:
# Refem-ho tot sense 'DepDelay'

## Variable dependent
y = vols_normalitzat['ArrDelay_delayed']

# Variables predictores
x = vols_normalitzat.drop(['ArrDelay_delayed', 'DepDelay'], axis = 1)

# LLista amb el nom dels predictors
x_columnes = list(x.columns)

In [68]:
# Creació de la mostra d'entrenament i validació
mostra_entrenament_normalitzada_senseDepDelay_x, mostra_validacio_normalitzada_senseDepDelay_x, mostra_entrenament_normalitzada_senseDepDelay_y, mostra_validacio_normalitzada_senseDepDelay_y = train_test_split(x, y, test_size = 0.2, random_state=567)

mostra_entrenament_normalitzada_senseDepDelay = pd.concat([mostra_entrenament_normalitzada_senseDepDelay_x, mostra_entrenament_normalitzada_senseDepDelay_y], axis=1)
mostra_validacio_normalitzada_senseDepDelay = pd.concat([mostra_validacio_normalitzada_senseDepDelay_x, mostra_validacio_normalitzada_senseDepDelay_y], axis=1)

print("Tamany de mostra d'entrenament: {}".format(mostra_entrenament_normalitzada_senseDepDelay_x.shape[0]))
print("Tamany de mostra de validació: {}".format(mostra_validacio_normalitzada_senseDepDelay_x.shape[0]))

display(mostra_entrenament_normalitzada_senseDepDelay)
display(mostra_entrenament_normalitzada_senseDepDelay_x.head())
display(mostra_entrenament_normalitzada_senseDepDelay_y.head())

display(mostra_validacio_normalitzada_senseDepDelay)
display(mostra_validacio_normalitzada_senseDepDelay_x.head())
display(mostra_validacio_normalitzada_senseDepDelay_y.head())

Tamany de mostra d'entrenament: 12000
Tamany de mostra de validació: 3000


,ActualElapsedTime,AirTime,Distance,TaxiIn,TaxiOut,ArrDelay_delayed
id,,,,,,
1020517,0.171429,0.154723,0.132225,0.072917,0.067308,1
751521,0.135338,0.122150,0.079903,0.145833,0.032051,0
1528665,0.142857,0.141694,0.115798,0.041667,0.041667,1
815631,0.445113,0.462541,0.542081,0.062500,0.048077,1
1169295,0.557895,0.516287,0.449605,0.062500,0.182692,1
...,...,...,...,...,...,...
3365651,0.482707,0.490228,0.466031,0.114583,0.057692,1
1214504,0.114286,0.112378,0.084364,0.031250,0.041667,1
886636,0.072180,0.055375,0.048672,0.052083,0.057692,1


,ActualElapsedTime,AirTime,Distance,TaxiIn,TaxiOut
id,,,,,
1020517,0.171429,0.154723,0.132225,0.072917,0.067308
751521,0.135338,0.122150,0.079903,0.145833,0.032051
1528665,0.142857,0.141694,0.115798,0.041667,0.041667
815631,0.445113,0.462541,0.542081,0.062500,0.048077
1169295,0.557895,0.516287,0.449605,0.062500,0.182692


id
1020517    1
751521     0
1528665    1
815631     1
1169295    1
Name: ArrDelay_delayed, dtype: int32

,ActualElapsedTime,AirTime,Distance,TaxiIn,TaxiOut,ArrDelay_delayed
id,,,,,,
689285,0.357895,0.351792,0.404786,0.052083,0.083333,1
1513378,0.066165,0.042345,0.035895,0.052083,0.070513,0
80624,0.114286,0.105863,0.074630,0.052083,0.048077,1
1239004,0.064662,0.066775,0.053133,0.041667,0.022436,1
1327794,0.066165,0.052117,0.018252,0.062500,0.048077,1
...,...,...,...,...,...,...
6618180,0.064662,0.030945,0.013182,0.041667,0.092949,1
5023448,0.055639,0.045603,0.024133,0.052083,0.041667,1
649241,0.187970,0.193811,0.152099,0.062500,0.028846,1


,ActualElapsedTime,AirTime,Distance,TaxiIn,TaxiOut
id,,,,,
689285,0.357895,0.351792,0.404786,0.052083,0.083333
1513378,0.066165,0.042345,0.035895,0.052083,0.070513
80624,0.114286,0.105863,0.074630,0.052083,0.048077
1239004,0.064662,0.066775,0.053133,0.041667,0.022436
1327794,0.066165,0.052117,0.018252,0.062500,0.048077


id
689285     1
1513378    0
80624      1
1239004    1
1327794    1
Name: ArrDelay_delayed, dtype: int32

In [69]:
# Model de xarxes neuronals
xarxa_neuronal_normalitzada_senseDepDelay = MLPClassifier(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=500, random_state=5678)
xarxa_neuronal_normalitzada_senseDepDelay.fit(mostra_entrenament_normalitzada_senseDepDelay_x, mostra_entrenament_normalitzada_senseDepDelay_y)

# Generem la predicció
xarxa_neuronal_normalitzada_senseDepDelay_prediccio = xarxa_neuronal_normalitzada_senseDepDelay.predict(mostra_validacio_normalitzada_senseDepDelay_x)

In [70]:
# Model logístic
logistica_normalitzada_senseDepDelay = LogisticRegression(max_iter=1000, random_state=5678)
logistica_normalitzada_senseDepDelay.fit(mostra_entrenament_normalitzada_senseDepDelay_x, mostra_entrenament_normalitzada_senseDepDelay_y)

# Generem la predicció
logistica_normalitzada_senseDepDelay_prediccio = logistica_normalitzada_senseDepDelay.predict(mostra_validacio_normalitzada_senseDepDelay_x)

In [71]:
# Model xgboost_normalitzada_senseDepDelay
xgboost_normalitzada_senseDepDelay = xgb.XGBClassifier(objective ='binary:logistic', use_label_encoder=False, random_state=5678)
xgboost_normalitzada_senseDepDelay.fit(mostra_entrenament_normalitzada_senseDepDelay_x, mostra_entrenament_normalitzada_senseDepDelay_y)

# Generem la predicció
xgboost_normalitzada_senseDepDelay_prediccio = xgboost_normalitzada_senseDepDelay.predict(mostra_validacio_normalitzada_senseDepDelay_x)

[12:53:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Comparativa dels models de classificació utilitzant la precisió (accuracy), una matriu de confiança i d’altres mètriques més avançades.

In [72]:
# Calculem el grau de exactitud

# Model de xarxes neuronals
xarxa_neuronal_normalitzada_senseDepDelay_exactitud = round(metrics.accuracy_score(mostra_validacio_normalitzada_senseDepDelay_y, xarxa_neuronal_normalitzada_senseDepDelay_prediccio),4)
print(xarxa_neuronal_normalitzada_senseDepDelay_exactitud)

# Model logístic
logistica_normalitzada_senseDepDelay_exactitud = round(metrics.accuracy_score(mostra_validacio_normalitzada_senseDepDelay_y, logistica_normalitzada_senseDepDelay_prediccio),4)
print(logistica_normalitzada_senseDepDelay_exactitud)

# Model xgboost_normalitzada_senseDepDelay
xgboost_normalitzada_senseDepDelay_exactitud = round(metrics.accuracy_score(mostra_validacio_normalitzada_senseDepDelay_y, xgboost_normalitzada_senseDepDelay_prediccio),4)
print(xgboost_normalitzada_senseDepDelay_exactitud)

0.9037
0.903
0.8973


El model més exacte és el de xarxes_neuronals

In [73]:
# Calculem la matriu de confusió

# Model de xarxes neuronals
xarxa_neuronal_normalitzada_senseDepDelay_matriu_confusio = metrics.confusion_matrix(mostra_validacio_normalitzada_senseDepDelay_y, xarxa_neuronal_normalitzada_senseDepDelay_prediccio)
print(xarxa_neuronal_normalitzada_senseDepDelay_matriu_confusio)

# Model logístic
logistica_normalitzada_senseDepDelay_matriu_confusio = metrics.confusion_matrix(mostra_validacio_normalitzada_senseDepDelay_y, logistica_normalitzada_senseDepDelay_prediccio)
print(logistica_normalitzada_senseDepDelay_matriu_confusio)

# Model xgboost_normalitzada_senseDepDelay
xgboost_normalitzada_senseDepDelay_matriu_confusio = metrics.confusion_matrix(mostra_validacio_normalitzada_senseDepDelay_y, xgboost_normalitzada_senseDepDelay_prediccio)
print(xgboost_normalitzada_senseDepDelay_matriu_confusio)

[[   0  289]
 [   0 2711]]
[[   0  289]
 [   2 2709]]
[[  11  278]
 [  30 2681]]


El model amb una matriu de confusió amb més verdaders positius i verdaders negatius és el de xgboost.

In [74]:
# Calculem el grau de precisió

# Model de xarxes neuronals
xarxa_neuronal_normalitzada_senseDepDelay_precisio = round(metrics.precision_score(mostra_validacio_normalitzada_senseDepDelay_y, xarxa_neuronal_normalitzada_senseDepDelay_prediccio),4)
print(xarxa_neuronal_normalitzada_senseDepDelay_precisio)

# Model logístic
logistica_normalitzada_senseDepDelay_precisio = round(metrics.precision_score(mostra_validacio_normalitzada_senseDepDelay_y, logistica_normalitzada_senseDepDelay_prediccio),4)
print(logistica_normalitzada_senseDepDelay_precisio)

# Model xgboost_normalitzada_senseDepDelay
xgboost_normalitzada_senseDepDelay_precisio = round(metrics.precision_score(mostra_validacio_normalitzada_senseDepDelay_y, xgboost_normalitzada_senseDepDelay_prediccio),4)
print(xgboost_normalitzada_senseDepDelay_precisio)

0.9037
0.9036
0.906


El model més precís és el xgboost

In [75]:
# Calculem l'àrea sota la corba

# Model de xarxes neuronals
xarxa_neuronal_normalitzada_senseDepDelay_asc = round(metrics.roc_auc_score(mostra_validacio_normalitzada_senseDepDelay_y, xarxa_neuronal_normalitzada_senseDepDelay_prediccio),4)
print(xarxa_neuronal_normalitzada_senseDepDelay_asc)

# Model logístic
logistica_normalitzada_senseDepDelay_asc = round(metrics.roc_auc_score(mostra_validacio_normalitzada_senseDepDelay_y, logistica_normalitzada_senseDepDelay_prediccio),4)
print(logistica_normalitzada_senseDepDelay_asc)

# Model xgboost_normalitzada_senseDepDelay
xgboost_normalitzada_senseDepDelay_asc = round(metrics.roc_auc_score(mostra_validacio_normalitzada_senseDepDelay_y, xgboost_normalitzada_senseDepDelay_prediccio),4)
print(xgboost_normalitzada_senseDepDelay_asc)

0.5
0.4996
0.5135


El model amb una àrea sota la corba és el xgboost

Entrena’ls utilitzant els diferents paràmetres que admeten.

In [76]:
# Model xarxa neuronal

# Definició dels paràmetres a afinar
xarxa_neuronal_normalitzada_senseDepDelay_parametres_a_afinar = {'hidden_layer_sizes': [[8,8],[8,8,8],[8,8,8,8]], 'activation': ['identity', 'logistic', 'tanh', 'relu'],'solver': ['lbfgs', 'sgd', 'adam'],}

# Métrica escollida d'entre: precision, recall, f1, f1_weighted, f1_micro, f1_macro, accuracy, roc_auc
xarxa_neuronal_normalitzada_senseDepDelay_metrica = 'accuracy'

# Entrenament dels hiper_paràmetres de la métrica escollida
xarxa_neuronal_normalitzada_senseDepDelay_entrenament = GridSearchCV(estimator = MLPClassifier(max_iter=500,random_state=5678), param_grid = xarxa_neuronal_normalitzada_senseDepDelay_parametres_a_afinar, cv=10, scoring='%s' % xarxa_neuronal_normalitzada_senseDepDelay_metrica, n_jobs= -1)
# Ajust
xarxa_neuronal_normalitzada_senseDepDelay_entrenament.fit(mostra_entrenament_normalitzada_senseDepDelay_x, mostra_entrenament_normalitzada_senseDepDelay_y)
    
print("Els paràmetres que afinen millor la métrica són: \n{}\n".format(xarxa_neuronal_normalitzada_senseDepDelay_entrenament.best_params_))

xarxa_neuronal_normalitzada_senseDepDelay_metrica_mitjana = xarxa_neuronal_normalitzada_senseDepDelay_entrenament.cv_results_['mean_test_score']
xarxa_neuronal_normalitzada_senseDepDelay_metrica_desviacio_estandar = xarxa_neuronal_normalitzada_senseDepDelay_entrenament.cv_results_['std_test_score']

Els paràmetres que afinen millor la métrica són: 
{'activation': 'tanh', 'hidden_layer_sizes': [8, 8], 'solver': 'adam'}



In [77]:
print("Llista de puntuacións obtinguda en les diferents combinacions de hiper_paràmetres:\n")
print("Puntuació   Desv. Estd.     Paràmetres")
print("====================================================================================================")

for mitjana, desviacio_estandard, parametres in zip(np.sort(xarxa_neuronal_normalitzada_senseDepDelay_metrica_mitjana)[::-1], xarxa_neuronal_normalitzada_senseDepDelay_metrica_desviacio_estandar, xarxa_neuronal_normalitzada_senseDepDelay_entrenament.cv_results_['params']):
    print("{}   |  [+/-{}]  |  {}".format(round(mitjana,4), round(desviacio_estandard,4), parametres))
    print("____________________________________________________________________________________________________")

Llista de puntuacións obtinguda en les diferents combinacions de hiper_paràmetres:

Puntuació   Desv. Estd.     Paràmetres
0.8943   |  [+/-0.0029]  |  {'activation': 'identity', 'hidden_layer_sizes': [8, 8], 'solver': 'lbfgs'}
____________________________________________________________________________________________________
0.8941   |  [+/-0.0003]  |  {'activation': 'identity', 'hidden_layer_sizes': [8, 8], 'solver': 'sgd'}
____________________________________________________________________________________________________
0.8941   |  [+/-0.0023]  |  {'activation': 'identity', 'hidden_layer_sizes': [8, 8], 'solver': 'adam'}
____________________________________________________________________________________________________
0.8941   |  [+/-0.0029]  |  {'activation': 'identity', 'hidden_layer_sizes': [8, 8, 8], 'solver': 'lbfgs'}
____________________________________________________________________________________________________
0.8941   |  [+/-0.0003]  |  {'activation': 'identity', 'h

In [78]:
# Model logístic

# Definició dels paràmetres a afinar
logistica_normalitzada_senseDepDelay_parametres_a_afinar = {'penalty': ['l2', 'none'],'C': [0.5,1.0,1.5],'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],}

# Métrica escollida d'entre: precision, recall, f1, f1_weighted, f1_micro, f1_macro, accuracy, roc_auc
logistica_normalitzada_senseDepDelay_metrica = 'accuracy'

# Entrenament dels hiper_paràmetres de la métrica escollida
logistica_normalitzada_senseDepDelay_entrenament = GridSearchCV(estimator = LogisticRegression(max_iter = 1000), param_grid = logistica_normalitzada_senseDepDelay_parametres_a_afinar, cv=10, scoring='%s' % logistica_normalitzada_senseDepDelay_metrica, n_jobs= -1)
# Ajust
logistica_normalitzada_senseDepDelay_entrenament.fit(mostra_entrenament_normalitzada_senseDepDelay_x, mostra_entrenament_normalitzada_senseDepDelay_y)
    
print("Els paràmetres que afinen millor la métrica són: \n{}\n".format(logistica_normalitzada_senseDepDelay_entrenament.best_params_))

logistica_normalitzada_senseDepDelay_metrica_mitjana = logistica_normalitzada_senseDepDelay_entrenament.cv_results_['mean_test_score']
logistica_normalitzada_senseDepDelay_metrica_desviacio_estandar = logistica_normalitzada_senseDepDelay_entrenament.cv_results_['std_test_score']

Els paràmetres que afinen millor la métrica són: 
{'C': 0.5, 'penalty': 'l2', 'solver': 'newton-cg'}



In [79]:
print("Llista de puntuacions obtinguda en les diferents combinacions de hiper_paràmetres:\n")
print("Puntuació   Desv. Estd.     Paràmetres")
print("====================================================================================================")

for mitjana, desviacio_estandard, parametres in zip(np.sort(logistica_normalitzada_senseDepDelay_metrica_mitjana)[::-1], logistica_normalitzada_senseDepDelay_metrica_desviacio_estandar, logistica_normalitzada_senseDepDelay_entrenament.cv_results_['params']):
    print("{}   |  [+/-{}]  |  {}".format(round(mitjana,4), round(desviacio_estandard,4), parametres))
    print("____________________________________________________________________________________________________")

Llista de puntuacions obtinguda en les diferents combinacions de hiper_paràmetres:

Puntuació   Desv. Estd.     Paràmetres
0.8938   |  [+/-0.0006]  |  {'C': 0.5, 'penalty': 'l2', 'solver': 'newton-cg'}
____________________________________________________________________________________________________
0.8938   |  [+/-0.0006]  |  {'C': 0.5, 'penalty': 'l2', 'solver': 'lbfgs'}
____________________________________________________________________________________________________
0.8938   |  [+/-0.0006]  |  {'C': 0.5, 'penalty': 'l2', 'solver': 'sag'}
____________________________________________________________________________________________________
0.8938   |  [+/-0.0006]  |  {'C': 0.5, 'penalty': 'l2', 'solver': 'saga'}
____________________________________________________________________________________________________
0.8936   |  [+/-0.0029]  |  {'C': 0.5, 'penalty': 'none', 'solver': 'newton-cg'}
___________________________________________________________________________________________

In [80]:
# Model xgboost_normalitzada_senseDepDelay

# Definició dels paràmetres a afinar
xgboost_normalitzada_senseDepDelay_parametres_a_afinar = {'booster': ['gbtree', 'gblinear', 'dart'],}

# Métrica escollida d'entre: precision, recall, f1, f1_weighted, f1_micro, f1_macro, accuracy, roc_auc
xgboost_normalitzada_senseDepDelay_metrica = 'accuracy'

# Entrenament dels hiper_paràmetres de la métrica escollida
xgboost_normalitzada_senseDepDelay_entrenament = GridSearchCV(estimator = xgb.XGBClassifier(objective ='binary:logistic', use_label_encoder = False, random_state = 5678), param_grid = xgboost_normalitzada_senseDepDelay_parametres_a_afinar, cv=10, scoring='%s' % xgboost_normalitzada_senseDepDelay_metrica, n_jobs= -1)
# Ajust
xgboost_normalitzada_senseDepDelay_entrenament.fit(mostra_entrenament_normalitzada_senseDepDelay_x, mostra_entrenament_normalitzada_senseDepDelay_y)
    
print("Els paràmetres que afinen millor la métrica són: \n{}\n".format(xgboost_normalitzada_senseDepDelay_entrenament.best_params_))

xgboost_normalitzada_senseDepDelay_metrica_mitjana = xgboost_normalitzada_senseDepDelay_entrenament.cv_results_['mean_test_score']
xgboost_normalitzada_senseDepDelay_metrica_desviacio_estandar = xgboost_normalitzada_senseDepDelay_entrenament.cv_results_['std_test_score']

[13:02:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Els paràmetres que afinen millor la métrica són: 
{'booster': 'gblinear'}



In [81]:
print("Llista de puntuacions obtinguda en les diferents combinacions de hiper_paràmetres:\n")
print("Puntuació   Desv. Estd.     Paràmetres")
print("====================================================================================================")

for mitjana, desviacio_estandard, parametres in zip(np.sort(xgboost_normalitzada_senseDepDelay_metrica_mitjana)[::-1], xgboost_normalitzada_senseDepDelay_metrica_desviacio_estandar, xgboost_normalitzada_senseDepDelay_entrenament.cv_results_['params']):
    print("{}   |  [+/-{}]  |  {}".format(round(mitjana,4), round(desviacio_estandard,4), parametres))
    print("____________________________________________________________________________________________________")

Llista de puntuacions obtinguda en les diferents combinacions de hiper_paràmetres:

Puntuació   Desv. Estd.     Paràmetres
0.8932   |  [+/-0.004]  |  {'booster': 'gbtree'}
____________________________________________________________________________________________________
0.8869   |  [+/-0.0008]  |  {'booster': 'gblinear'}
____________________________________________________________________________________________________
0.8869   |  [+/-0.004]  |  {'booster': 'dart'}
____________________________________________________________________________________________________


Compara el seu rendiment utilitzant l’aproximació traint/test o cross-validation

In [82]:
# Model xarxa neuronal

xarxa_neuronal_normalitzada_senseDepDelay_prediccio_rendiment = xarxa_neuronal_normalitzada_senseDepDelay.predict(mostra_validacio_normalitzada_senseDepDelay_x)
xarxa_neuronal_normalitzada_senseDepDelay_metrica_exactitud_prediccio_rendiment = round(metrics.accuracy_score(mostra_validacio_normalitzada_senseDepDelay_y, xarxa_neuronal_normalitzada_senseDepDelay_prediccio_rendiment),4)
xarxa_neuronal_normalitzada_senseDepDelay_metrica_asc_prediccio_rendiment = round(metrics.roc_auc_score(mostra_validacio_normalitzada_senseDepDelay_y, xarxa_neuronal_normalitzada_senseDepDelay_prediccio_rendiment),4)

#print(classification_report(mostra_validacio_normalitzada_senseDepDelay_y, xarxa_neuronal_normalitzada_senseDepDelay_prediccio_rendiment))
print("Exactitud:           ", xarxa_neuronal_normalitzada_senseDepDelay_metrica_exactitud_prediccio_rendiment)
print("Àrea sota la corba  :", xarxa_neuronal_normalitzada_senseDepDelay_metrica_asc_prediccio_rendiment)

Exactitud:            0.9037
Àrea sota la corba  : 0.5


In [83]:
# Model logístic

logistica_normalitzada_senseDepDelay_prediccio_rendiment = logistica_normalitzada_senseDepDelay.predict(mostra_validacio_normalitzada_senseDepDelay_x)
logistica_normalitzada_senseDepDelay_metrica_exactitud_prediccio_rendiment = round(metrics.accuracy_score(mostra_validacio_normalitzada_senseDepDelay_y, logistica_normalitzada_senseDepDelay_prediccio_rendiment),4)
logistica_normalitzada_senseDepDelay_metrica_asc_prediccio_rendiment = round(metrics.roc_auc_score(mostra_validacio_normalitzada_senseDepDelay_y, logistica_normalitzada_senseDepDelay_prediccio_rendiment),4)

#print(classification_report(mostra_validacio_normalitzada_senseDepDelay_y, logistica_normalitzada_senseDepDelay_prediccio_rendiment))
print("Exactitud:           ", logistica_normalitzada_senseDepDelay_metrica_exactitud_prediccio_rendiment)
print("Àrea sota la corba  :", logistica_normalitzada_senseDepDelay_metrica_asc_prediccio_rendiment)

Exactitud:            0.903
Àrea sota la corba  : 0.4996


In [84]:
# Model xgboost_normalitzada_senseDepDelay

xgboost_normalitzada_senseDepDelay_prediccio_rendiment = xgboost_normalitzada_senseDepDelay.predict(mostra_validacio_normalitzada_senseDepDelay_x)
xgboost_normalitzada_senseDepDelay_metrica_exactitud_prediccio_rendiment = round(metrics.accuracy_score(mostra_validacio_normalitzada_senseDepDelay_y, xgboost_normalitzada_senseDepDelay_prediccio_rendiment),4)
xgboost_normalitzada_senseDepDelay_metrica_asc_prediccio_rendiment = round(metrics.roc_auc_score(mostra_validacio_normalitzada_senseDepDelay_y, xgboost_normalitzada_senseDepDelay_prediccio_rendiment),4)

#print(classification_report(mostra_validacio_normalitzada_senseDepDelay_y, xgboost_normalitzada_senseDepDelay_prediccio_rendiment))
print("Exactitud:           ", xgboost_normalitzada_senseDepDelay_metrica_exactitud_prediccio_rendiment)
print("Àrea sota la corba:  ", xgboost_normalitzada_senseDepDelay_metrica_asc_prediccio_rendiment)

Exactitud:            0.8973
Àrea sota la corba:   0.5135
